<a href="https://colab.research.google.com/github/anyalitica/alteryx-api-exchange-rates/blob/main/nasa_close_approaches_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries used

import requests
import pandas as pd
from datetime import datetime, date, timedelta


In [ ]:
api_key = 'S5eW3OAAq6lzTV4kDlxxzW45ichN3FyLgwzsdOmE'

Get NEOs close to earth from today and 7 days ahead

In [ ]:
# Set start date to be today
start_date = datetime.today()

# And end date 7 days from today
end_date = start_date + timedelta(days=7)

In [ ]:
# convert start and end dates to strings for the URL
start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

In [ ]:
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={api_key}'
url

'https://api.nasa.gov/neo/rest/v1/feed?start_date=2022-11-22&end_date=2022-11-29&api_key=S5eW3OAAq6lzTV4kDlxxzW45ichN3FyLgwzsdOmE'

In [ ]:
response_code = requests.get(url)
response_code

<Response [200]>

In [ ]:
response = response_code.json()
response

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2022-11-29&end_date=2022-12-06&detailed=false&api_key=S5eW3OAAq6lzTV4kDlxxzW45ichN3FyLgwzsdOmE',
  'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2022-11-15&end_date=2022-11-22&detailed=false&api_key=S5eW3OAAq6lzTV4kDlxxzW45ichN3FyLgwzsdOmE',
  'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2022-11-22&end_date=2022-11-29&detailed=false&api_key=S5eW3OAAq6lzTV4kDlxxzW45ichN3FyLgwzsdOmE'},
 'element_count': 63,
 'near_earth_objects': {'2022-11-26': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2496817?api_key=S5eW3OAAq6lzTV4kDlxxzW45ichN3FyLgwzsdOmE'},
    'id': '2496817',
    'neo_reference_id': '2496817',
    'name': '496817 (1989 VB)',
    'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2496817',
    'absolute_magnitude_h': 19.9,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.2783267681,
      'estimated_diameter_max': 0.6223575734},
     'meters': {'esti

In [ ]:
# iterate through every day and every object data to get the elements, update the dataframe

# The term LD (Lunar Distance) refers to the average distance between the Earth and Moon. 
# For data reported on this site, we use a mean semimajor axis for the moon of 384400 km (~.002570 au) to define one LD.

dataframe = pd.DataFrame(columns = ['id'
                                    , 'name'
                                    , 'nasa_jpl_url'
                                    , 'absolute_magnitude_h'
                                    , 'estimated_diameter_min_meters'
                                    , 'estimated_diameter_max_meters'
                                    , 'is_potentially_hazardous_asteroid'
                                    , 'is_sentry_object'
                                    , 'close_approach_datetime'
                                    , 'relative_velocity_km_sec'
                                    , 'relative_velocity_km_hour'
                                    , 'miss_distance_astronomical'
                                    , 'miss_distance_lunar'
                                    , 'miss_distance_km'
                                    , 'orbiting_body'])

for date in response['near_earth_objects']:
  for i in response['near_earth_objects'][date]:
    id = i['id']
    name = i['name']
    nasa_jpl_url = i['nasa_jpl_url']
    absolute_magnitude_h = i['absolute_magnitude_h']
    estimated_diameter_min_meters = i['estimated_diameter']['meters']['estimated_diameter_min']
    estimated_diameter_max_meters = i['estimated_diameter']['meters']['estimated_diameter_max']
    is_potentially_hazardous_asteroid = i['is_potentially_hazardous_asteroid']
    is_sentry_object = i['is_sentry_object']
    # Convert close approach date time from epoch to UNIX format
    close_approach_epoch = i['close_approach_data'][0]['epoch_date_close_approach']
    close_approach_datetime = datetime.fromtimestamp(close_approach_epoch/1000).strftime('%Y-%m-%d %H:%M:%S')
    relative_velocity_km_sec = i['close_approach_data'][0]['relative_velocity']['kilometers_per_second']
    relative_velocity_km_hour = i['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']
    miss_distance_astronomical = i['close_approach_data'][0]['miss_distance']['astronomical']
    miss_distance_lunar = i['close_approach_data'][0]['miss_distance']['lunar']
    miss_distance_km = i['close_approach_data'][0]['miss_distance']['kilometers']
    orbiting_body = i['close_approach_data'][0]['orbiting_body']


    dataframe_date = pd.DataFrame([[ id
                                    , name
                                    , nasa_jpl_url
                                    , absolute_magnitude_h
                                    , estimated_diameter_min_meters
                                    , estimated_diameter_max_meters
                                    , is_potentially_hazardous_asteroid
                                    , is_sentry_object
                                    , close_approach_datetime
                                    , relative_velocity_km_sec
                                    , relative_velocity_km_hour
                                    , miss_distance_astronomical
                                    , miss_distance_lunar
                                    , miss_distance_km
                                    , orbiting_body]],
                                  columns=[
                                    'id'
                                    ,'name'
                                    ,'nasa_jpl_url'
                                    ,'absolute_magnitude_h'
                                    , 'estimated_diameter_min_meters'
                                    , 'estimated_diameter_max_meters'
                                    , 'is_potentially_hazardous_asteroid'
                                    , 'is_sentry_object'
                                    , 'close_approach_datetime'
                                    , 'relative_velocity_km_sec'
                                    , 'relative_velocity_km_hour'
                                    , 'miss_distance_astronomical'
                                    , 'miss_distance_lunar'
                                    , 'miss_distance_km'
                                    , 'orbiting_body'])
    dataframe = pd.concat([dataframe,dataframe_date])


In [ ]:
dataframe_final = dataframe.reset_index(drop=True)
dataframe_final.head(10)

,id,name,nasa_jpl_url,absolute_magnitude_h,estimated_diameter_min_meters,estimated_diameter_max_meters,is_potentially_hazardous_asteroid,is_sentry_object,close_approach_datetime,relative_velocity_km_sec,relative_velocity_km_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_km,orbiting_body
0,2496817,496817 (1989 VB),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2496817,19.90,278.326768,622.357573,True,False,2022-11-26 07:20:00,19.7172994245,70982.2779281939,0.4305875942,167.4985741438,64414986.940744354,Earth
1,2509102,509102 (2005 VB7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2509102,19.60,319.561887,714.562102,False,False,2022-11-26 11:58:00,18.6942534677,67299.3124836934,0.2910875208,113.2330455912,43546073.095260696,Earth
2,3363699,(2006 XR4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3363699,26.20,15.295194,34.201092,False,False,2022-11-26 03:29:00,23.1302804307,83269.009550578,0.4705113787,183.0289263143,70387500.064283369,Earth
3,3514549,(2010 FA81),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3514549,22.40,88.014652,196.806745,False,False,2022-11-26 04:10:00,8.9736050897,32304.9783230178,0.3079306344,119.7850167816,46065767.013988728,Earth
4,3781947,(2017 SX17),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3781947,28.30,5.815070,13.002893,False,False,2022-11-26 01:59:00,5.5233422688,19884.0321675892,0.1205561138,46.8963282682,18034937.839957606,Earth
5,54144299,(2021 JU3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54144299,27.09,10.152079,22.700738,False,False,2022-11-26 09:38:00,19.1343590201,68883.6924723928,0.2657334429,103.3703092881,39753157.045606623,Earth
6,54212682,(2021 UO1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54212682,28.03,6.584988,14.724480,False,False,2022-11-26 05:58:00,14.0591836324,50613.0610765522,0.4527046696,176.1021164744,67723654.311213752,Earth
7,54269054,(2022 GM1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54269054,25.56,20.537850,45.924029,False,False,2022-11-26 22:15:00,5.4719283928,19698.942214251,0.1746659039,67.9450366171,26129647.185064693,Earth
8,2487580,487580 (2015 BA92),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2487580,20.13,250.354255,559.809132,False,False,2022-11-27 01:22:00,27.0158314356,97256.993168164,0.3831029354,149.0270418706,57311383.126587598,Earth
9,3695735,(2014 UJ192),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3695735,21.91,110.294785,246.626636,False,False,2022-11-27 09:03:00,8.9706783675,32294.4421231339,0.4195527731,163.2060287359,62764201.208353297,Earth


In [ ]:
# convert the dataframe to list to pass back to Tableau

data = dataframe_final.to_dict(orient="list")
data

{'id': ['2496817',
  '2509102',
  '3363699',
  '3514549',
  '3781947',
  '54144299',
  '54212682',
  '54269054',
  '2487580',
  '3695735',
  '3831464',
  '3888756',
  '3892803',
  '54017044',
  '2345705',
  '2446924',
  '3167340',
  '3670290',
  '3746497',
  '3825088',
  '3826775',
  '3893274',
  '3893770',
  '3986725',
  '54087668',
  '54268442',
  '54269647',
  '2036183',
  '3262792',
  '3299722',
  '3561033',
  '3561034',
  '3599929',
  '54099637',
  '54224159',
  '54224419',
  '2152563',
  '3163741',
  '3304382',
  '3388311',
  '3478250',
  '3799681',
  '3824091',
  '3825017',
  '3395962',
  '3645236',
  '3738544',
  '3754548',
  '3836137',
  '3837977',
  '3842751',
  '54151572',
  '54225732',
  '2331876',
  '3163748',
  '3836249',
  '3843819',
  '54087415',
  '2458368',
  '3279432',
  '3614980',
  '3741151',
  '54016464'],
 'name': ['496817 (1989 VB)',
  '509102 (2005 VB7)',
  '(2006 XR4)',
  '(2010 FA81)',
  '(2017 SX17)',
  '(2021 JU3)',
  '(2021 UO1)',
  '(2022 GM1)',
  '487580